# 说明

创建于2024-3-11日，旨在用于提取脑电的微分熵的特征，其形状为31\*5的向量

区别于/home/yihaoyuan/WorkSpace/SpatialAbilityTest/ExtractFeatures.ipynb中形状为31\*150的特征

In [1]:
import torch
import numpy as np
import sys
import spatial_utils.data_process as sudp
import spatial_utils.data_read as sudr
import spatial_utils.feature_extract as sufe
# 好像不清楚滑窗截取的数据存在哪儿了，重新整一下
import tqdm
from datetime import datetime
import os
import pandas as pd
import time

In [2]:
subject_no = 1
eeg_trial_list = sudr.get_eeg_trials(subject_no, use_ICA=False) # 先不用ICA，有点慢，后面再说
label_list = sudr.get_labels(subject_no)
print(len(eeg_trial_list))
print(eeg_trial_list[0].shape)
print(label_list.shape)
print(label_list[:,-1])

Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject01/eeg/000001.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
54
(3859, 31)
(54, 3)
[1 4 4 1 4 4 4 1 1 1 3 1 3 4 2 1 2 3 4 2 1 1 4 4 2 1 4 1 4 4 4 3 2 2 1 1 4
 3 4 4 4 2 3 3 3 2 3 4 3 3 1 3 4 3]


In [ ]:
new_trails, new_labels = sudp.re_data_slide(trial=eeg_trial_list[0], label = label_list[0], win_len = 256, overlap = 0, is_filter = False, norm_method = None)
print(new_trails.shape, new_labels.shape)

In [3]:
eeg_trials = []
labels = []
# subject_lists = [i for i in range(1, 36) if i != 11 and i != 9 and i!= 19 and i!=20 and i!=33]
subject_lists = [1,2]
loop = tqdm.tqdm(subject_lists, total=len(subject_lists), leave=False)
for participant_no in loop:
    loop.set_description('Processing participant %d' % participant_no)
    eeg_trial_list = sudr.get_eeg_trials(participant_no)
    label_list = sudr.get_labels(participant_no)
    label_list = label_list[:, -1] - 1
    for ex_no in range(len(eeg_trial_list)):
        new_trails,new_labels = sudp.re_data_slide(trial=eeg_trial_list[ex_no], label = label_list[ex_no], win_len = 256, overlap = 0, is_filter = False, norm_method = None)   # 因为没有overlap，所以用1s大小更为通用一些
        eeg_trials.append(new_trails)
        labels.append(new_labels)
eeg_trials = np.concatenate(eeg_trials, axis=0)
labels = np.concatenate(labels, axis=0)
print(eeg_trials.shape, labels.shape)

Processing participant 1:   0%|          | 0/2 [00:00<?, ?it/s]

Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject01/eeg/000001.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']


Processing participant 2:  50%|█████     | 1/2 [00:06<00:06,  6.10s/it]

Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject02/eeg/000002.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']


(1696, 256, 31) (1696,)


In [5]:
# 计算DE特征
features = []
for trial in eeg_trials:
    feature = sufe.compute_DE(trial)
    features.append(feature)
features = np.array(features)
print(features.shape)
features.swapaxes(1,2).shape

(1696, 5, 31)


(1696, 31, 5)

In [17]:
eeg_trials.shape

(1696, 256, 31)

In [5]:
# 保存一下原始的全局label
subject_lists = [i for i in range(1, 36) if i != 11 and i != 9 and i!= 19 and i!=20 and i!=33]
labels = []
for participant_no in subject_lists:
    label_list = sudr.get_labels(participant_no)
    label_list = label_list[:, -1] - 1
    labels.append(label_list)
labels = np.concatenate(labels)
print(labels.shape)
np.save("/data/SpaticalTest/SlideData/total_33_labels.npy",labels)

(1620,)


## 保存一下脑电的每道题目的index

为了眼动和脑电数据量相匹配


In [2]:
import torch
import numpy as np
import sys
import spatial_utils.data_process as sudp
import spatial_utils.data_read as sudr
import spatial_utils.feature_extract as sufe

# 好像不清楚滑窗截取的数据存在哪儿了，重新整一下
import tqdm
from datetime import datetime
import os
import pandas as pd
import time
import tracemalloc
eeg_trials = []
labels = []
de_features = []
subject_indexs = [0]
eeg_ex_index = []
subject_lists = [
    i for i in range(1, 36) if i != 11 and i != 9 and i != 19 and i != 20 and i != 33
]

In [5]:
for participant_no in subject_lists:
    print(participant_no)
    eeg_trial_list = sudr.get_eeg_trials(participant_no, use_ICA=True)
    label_list = sudr.get_labels(participant_no)
    label_list = label_list[:, -1] - 1
    per_trial_index = [0]
    eeg_trials = []
    for ex_no in range(len(eeg_trial_list)):
        new_trails, new_labels = sudp.re_data_slide(
            trial=eeg_trial_list[ex_no],
            label=label_list[ex_no],
            win_len=256,
            overlap=0,
            is_filter=False,
            norm_method=None,
        )  # 因为没有overlap，所以用1s大小更为通用一些
        eeg_trials.extend(new_trails)
        per_trial_index.append(len(eeg_trials))
    eeg_ex_index.append(per_trial_index)

1
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject01/eeg/000001.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
2
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject02/eeg/000002.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
3
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject03/eeg/000003.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
4
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/old/subject04/eeg/000004.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5

/home/yihaoyuan/miniconda3/envs/torch/lib/python3.8/site-packages/sklearn/decomposition/_fastica.py:123: ConvergenceWarning: FastICA did not converge. Consider increasing tolerance or the maximum number of iterations.
  warnings.warn(


Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/202307/025/eeg/025.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
26
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/202307/026/eeg/026.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
27
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/202307/027/eeg/027.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
28
Finding 'sfreq' from header file: /data/SpaticalTest/SpaticalAbilityTest/202307/028/eeg/028.vhdr
Used Annotations descriptions: ['New Segment/', 'Stimulus/S  1', 'Stimulus/S  2', 'Stimulus/S  3', 'Stimulus/S  4', 'Stimulus/S  5', 'Stimulus/S  6']
29


In [9]:
test = np.array(eeg_ex_index)
np.save("/data/SpaticalTest/Spatial/total_33_eeg_ex_index.npy", test)

# 这里保存一下labels

In [7]:
import torch
import numpy as np
import sys
import spatial_utils.data_process as sudp
import spatial_utils.data_read as sudr
import spatial_utils.feature_extract as sufe

# 好像不清楚滑窗截取的数据存在哪儿了，重新整一下
import tqdm
from datetime import datetime
import os
import pandas as pd
import time

print(datetime.now())

subject_lists = [
    i
    for i in range(1, 36)
    if i != 11 and i != 9 and i != 19 and i != 20 and i != 33 and i != 15
]
print(len(subject_lists))
labels = []
for per_no in subject_lists:
    label = sudr.get_labels(per_no)
    labels.append(label[:,-1])
labels = np.concatenate(labels)
print(labels.shape)
np.save("/data/SpaticalTest/Spatial/labels29.npy", labels)

2024-04-10 14:09:05.813184
29
(1566,)
